In [1]:
import gurobipy as gp
from gurobipy import *
import itertools



the second argument is the number of clauses in the function
the third argument is the minimum length of each clause
prints a function whose first variable is maximally frequent, such that this frequency is as small
as possible,given the constraints
returns the frequency

In [2]:


def optimizeLen(numVars,currentLen,minClauseLen):
    #cartesian product [0,1]^n
    ListRep = list(itertools.product([0,1],repeat = numVars))

    m = gp.Model('HyperGraph')
    
    #each variable represents a subset
    x = m.addVars(ListRep,vtype = GRB.BINARY, name = "x")

    #decides if a is a subset of b
    def subset(a,b):
        if a==b :
            return False
        l = len(a)
        for i in range(l):
            if (a[i] > b[i]):
                return False
        return True

    #returns all subsets of a set (represented as a binary string of length n). not very efficient right now
    def subsets(hpBin):
        l = len(hpBin)
        res = []
        allSets = list(itertools.product([0,1],repeat = l))
        for hp in allSets:
            if subset(hp,hpBin):
                res.append(hp)
        return res

    #antichain constraints. ensures that no clause is a subset of another
    for i in ListRep:
        for j in subsets(i):
            m.addConstr(x[i] + x[j] <= 1, "antichain")
    m.addConstr(x.sum() == currentLen, "hpLength")

    
    #list of subsets that contains the nth element
    def hasElement(n):
        lst = []
        for s in x:
            if s[n]== 1:
                lst.append(s)
        return tupledict({key:x[key] for key in lst})
    
    hasOne = hasElement(0)

    
    for i in range(1,numVars):
        #these constraints ensure that the first variable has maximum frequency
        m.addConstr(hasOne.sum() - hasElement(i).sum() >= 0, "freq" + str(i+1))
        #and these ensure that all variables appear
        m.addConstr(hasElement(i).sum() >= 1, str(i) + "var")
    m.addConstr(hasOne.sum() >= 1, "1var")
    
    

    def subsetsOfLength(n):
        lst = []
        for s in x:
            if sum(s) == n :
                lst.append(s)
        return tupledict({key:x[key] for key in lst})


    for i in range(1,minClauseLen):
        m.addConstr(subsetsOfLength(i).sum() == 0, "minclause" + str(i))
    


    m.setObjective(x.sum(1, '*', '*', '*', '*', '*', '*', '*'), GRB.MINIMIZE)

    m.write('test.lp')

    m.optimize()

    def prettyprint(hpBin):
        res = []
        for i in range(len(hpBin)):
            if(hpBin[i] == 1):
                res.append(i+1)
        print("(", end="")
        for r in res:
            print(r, end = " ")
        print(")", end = " ")

    for i in ListRep:
        if (0.9 <= x[i].x <= 1.1):
            prettyprint(i)
                        
    obj = m.getObjective()
    return (obj.getValue())/currentLen

In [3]:
optimizeLen(8,4,2)

(6 8 ) (3 5 ) (2 7 ) (1 4 ) 

0.25

In [10]:
%%capture cap --no-stderr
for i in range(1,3):
    print("on ", i, " variables:")
    res = optimizeLen(8,i,2)
    print("")
    print("frequency = ", res)
    print("------------------------")
with open('output.txt', 'w') as f:
    f.write(cap.stdout)
cap.show()